In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.filters import maximum_filter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from skimage import measure

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
model = load_model('../input/cots-models/v5am2-004-0.674-0.167.h5')

In [ ]:
thr = 0.2
kernel = np.ones((7,7))
min_w=21
min_h=15
ratio = 1.6

model_input = keras.Input(shape=(None,None,3))
x = model_input/127.5-1.
x = keras.layers.Resizing(int(np.round(720*ratio)), int(np.round(1280*ratio)))(x)
x = model(x)
model = keras.Model(inputs=model_input, outputs=x)

for i, (image_np, sample_prediction_df) in enumerate(iter_test):

    p = model.predict(image_np[None, :,:,::-1])
    hm = p['hm'][0,:,:,0]
    reg = p['re'][0]

    hm[:3] = hm[:3]*.6
    hm[-3:] = hm[-3:]*.6
    hm[:,:3] = hm[:,:3]*.6
    hm[:,-3:] = hm[:,-3:]*.6
    
    local_maxi = hm*(hm == maximum_filter(hm,footprint=kernel))>thr
    py, px = np.where(local_maxi)
    conf = hm[py,px]
    bb = reg[py,px]
        
    px = px + 0.5 + bb[:,0] - bb[:,2]/2.
    py = py + 0.5 + bb[:,1] - bb[:,3]/2.
    ww = bb[:,2]
    hh = bb[:,3]
    
    bboxes = np.round(np.array([px,py,ww,hh]).T*8./ratio).astype('int')
    #bboxes = np.array([px,py,ww,hh]).T*8./ratio
    
    bboxes[:,2:] = bboxes[:,2:] + np.minimum(bboxes[:,:2], 0)
    bboxes[:,:2] = np.maximum(bboxes[:,:2], 0)
    bboxes[:,2] = np.minimum(bboxes[:,0]+bboxes[:,2], 1279)-bboxes[:,0]
    bboxes[:,3] = np.minimum(bboxes[:,1]+bboxes[:,3], 719)-bboxes[:,1]
    #valid= np.min(bboxes[:,2:], axis=1)>0
    valid= (bboxes[:,2]>(min_w/ratio))*(bboxes[:,3]>(min_h/ratio))
    bboxes = bboxes[valid]
    conf = conf[valid]
        
    predictions = []

    for j, (x,y,w,h) in enumerate(bboxes):
        predictions.append('{:.4f} {} {} {} {}'.format(conf[j], x, y, w, h))
        #predictions.append('{:.4f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(conf[j], x, y, w, h))

    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)
        
    if i<3:
        print('Prediction:', prediction_str)

In [ ]:
plt.imshow(hm)

In [ ]:
hm.max()